<p align="center">
  <img src="Graphics/Episode III.png" />
</p>

## (0) Classical Planning - Continued

### (0.1) Learning Outcomes

In this tutorial, we will cover:
* How do classical planning algorithms work?
* More classical planning problems in PDDL - Tower of Hanoi, Travelling Salesman Problem

## (1) Classical Planning Algorithms

**Reminder:** *Classical planning* is the problem of finding a sequence of actions for achieving a goal from an initial state assuming that actions have *deterministic* effects and that the environment is *fully observable*.

Now that we know how to represent classical planning problems, how can we go about actually *solving* them? 

We might begin by looking at the initial state and observing which actions can actually be applied to it (i.e. which actions have preconditions satisfied by the initial state). This will give us a set of new states, and for each of the new states we can discern which actions are *applicable* to that state, and we can continue the process for the new states we obtained from that and so on...

What are we building here? The *state-space* $S$ of our problem - the set of all possible states $s\in S$ that are reachable from the initial state $s_{0}$ by applying a sequence of deterministic actions $\{a_{i}\}, a\in A$ to $s_{0}$. For a goal $g$, there will be a subset $S_{g}	\subset S$ that contains all of the states $s$ that satisfy $g$ (assuming that such $s$ are actually reachable - otherwise there is no solution to our problem).


### (1.1) Classical Planning as Search
#### (1.1.1) State-Space Representation
Now that we have the concept of the state-space, we'd like to know how we can go about traversing this space betweem the initial state and the goal states. First we need to decide how to actually represent this space, and there are two common methods to do so:
* **Search Tree:** A directed acyclic graph whose root node is the initial state, whose branches represent the actions that take one state (the parent node) to another (the child node), and whose nodes are all states in the state-space (really, they represent the whole path taken to reach that state). 
* **State-Space Graph:** A graph (undirected or directed, cyclic or acyclic) whose nodes are all states in the state-space and whose edges represent the actions that take one state (node) to another (node). 

Let's see what they might look like for the simple problem we explored in the first part of the last tutorial:
<p align="center">
  <img src="Graphics/StateSpaceGraph_Vs_SearchTree.png" />
</p>

Here, we have a finite state-space graph (with 6 nodes and 5 edges), and an infinite search tree! If we allow the same state to be reached multiple times (i.e. we allow for *cycles*), then of course the tree will be infinite, even if the graph is finite. Does that mean that state-space graphs are better for this application?

Not necessarily! State-space graphs for complex problems are often way too large to build completely in our computer's memory (and we already saw that the same is true for search trees). So what do we do?

Focusing on search trees (which are used for this application more often), we will only build them partially and as necessary! We will only *expand* nodes in the tree when needed, and this will allow us to solve our problems in a much more computationally efficient manner. With such trees, we will represent nodes using a tuple $(\pi,s)$, where $\pi$ is the path from the root node to that node, and $s$ is the state represented by that node.

**Side Note:** When dealing with trees, we can also implement a method called *pruning* to remove nodes from the tree based on certain criteria. For example, we can conduct *cycle-checking*, which involves removing nodes $(\pi,s)$ that have an ancestor $(\pi',s')$ such that $s' = s$. In problems with finite state-spaces (i.e. classical planning problems), doing cycle-checking on our search tree will guarantee that our search terminates in a finite time.

#### (1.1.2) State-Space Search

Working with our new concepts for state-space representation, we finally arrive at the moment where we shall actually solve our planning problems. We will apply *search algorithms* (some of which you may have seen already in other courses) to our search trees in order to find a valid plan. There are two general approaches to state-space search:
* **Forward Search:** Start with the initial state as the root node and keep expanding nodes (according to some methodology) until one of the descendants which satisfies the goal is found (more common).
* **Backward Search:** Start with the goal state as the root node and keep expanding nodes (according to some methodology) until one of the descendants found is the initial state.

In the figure of the search tree shown above, we see that a forward search is being conducted, for example.

The tricky part here is deciding which node to expand at each stage, for which we can use strategies such as:
* **Breadth-First Search (BFS):** Explores all nodes at the present depth before moving on to the nodes at the next depth level (implemented using a First In First Out (FIFO) *queue* to keep track of the child nodes that were encountered but not yet explored).
<p align="center">
  <img src="Graphics/BFS.gif" alt="animated" />
</p>

* **Depth-First Search (DFS):** Explores as far as possible along each branch before backtracking, uses a Last In First Out (LIFO) *stack* to keep track of the nodes discovered so far along a specified branch (which helps in backtracking of the graph).
<p align="center">
  <img src="Graphics/DFS.gif" alt="animated" />
</p>

* **Uniform-Cost Search (UCS):** Explores the nodes with the lowest cumulative *cost* first, using a priority queue.
<p align="center">
  <img src="Graphics/UCS.gif" alt="animated" />
</p>

The cost that was mentioned in the last option is the cost of the action that takes us from a parent node to its child node, for example the cost of travelling from point A to point B might be evaluated using the Euclidean distance between them. The UCS algorithm expands first the nodes which have the lowest cumulative cost, i.e. the sum of the costs of all of the actions necessary to reach that node from the root node.

There are three important properties of such algorithms that we should consider:
* **Complexity:** How much time and space would the algorithm require (in the worst-case)?
* **Completeness:** Is the algorithm guaranteed to find a solution if one exists?
* **Optimality:** Is the algorithm guaranteed to find the least cost path?


##### **Complexity**

Let $b$ be the *branching factor* of our search tree (the number of children at each node - if this value is not uniform, an average branching factor can be calculated) and $m$ be the maximum depth of the tree (can be infinite). The number of nodes in an entire search tree will thus be $1+b+b^{2}+...+b^{m}=O(b^{m})$.

(1) Letting $s$ be the depth of the shallowest solution, then the time and space complexity of BFS will be $O(b^{s})$.

(2) DFS could process the whole tree, and so if $m$ is finite then the time complexity will be $O(b^{m})$. The space complexity is $O(bm)$, since the stack only stores nodes in the current path and their siblings (nodes on the same tier).

(3) Letting the solution's path have a total costs of $C$ and the minimal cost for a single edge (not its whole path) be $\epsilon$, then we can define an "effective depth" for the tree: $C/\epsilon$. Thus, the time and space complexity of UCS are $O(b^{C/\epsilon})$.

##### **Completeness**

(1) BFS will always find a solution if one exists (assuming $s$ is finite)!

(2) DFS is only complete if $m$ is finite, if not we can call the algorithm with a bound $L$ on the maximum depth explored (this is the DFS-L algorithm)

(3) Assuming the best solution has a finite cost, UCS is complete (in fact, it will always find the best solution --> optimal!)

##### **Optimality**

(1) BFS is only optimal if the cost of every edge in the tree is 1 (i.e. the cost of a path is equal to the number of edges/actions it contains) - otherwise not optimal

(2) DFS is *not* optimal - it will just choose the "leftmost" (imagine that we explore the tree from left to right) solution it can find

(3) UCS always finds the path with the optimal cost!

#### (1.1.3) Heuristic Search

The algorithms we've explored up until now are *uninformed* - they don't have any idea of where the goal(s) actually are. Sometimes, though, we can create a *heuristic function* that will estimate how close a state is to a goal state. Unlike before, this heuristic will depend on the particular search problem we are looking at (though this doesn't mean we can't apply them to domain-independent planning, more on this soon). For example, if we are trying to find a path from Haifa to Tel-Aviv, Euclidean distance could be a heuristic that we will tell us how far we are from Tel-Aviv at any given moment once we leave Haifa.

We can then propose a *greedy* search algorithm for traversing a search tree: we first expand nodes with the smallest heuristic values. Such an algorithm is complete (for a finite state-space), yet still *not* optimal. Its time and space complexity will depend on the heuristic - it could be much larger or smaller than BFS.
<p align="center">
  <img src="Graphics/Greedy.gif" alt="animated" width=500 />
</p>

We see that UCS searches in order of path cost $g(n)$, whereas greedy search orders by the proximity to the goal $h(n)$. How can we combine these concepts?

This is exactly what **A\* Search** does: it orders according to the sum $f(n) = g(n) + h(n)$. Is this complete? Optimal?

First we must define an *admissible* heuristic: it does *not* overestimate the distance to the goal, or more formally $\forall x, 0\leq h(x) \leq h^{*}(x)$, where $h^{*}(x)$ is the true cost to the nearest goal.

A* is complete for solvable problems, and if we use an admissible heuristic then A* is also optimal (assuming a finite state-space)! What's the drawback? The time and space complexity are $O(b^{m})$ - we have to explore all nodes in the tree and store them in memory. Even so, A* is still one of the most commonly used search algorithms in AI and robotics today!
<p align="center">
  <img src="Graphics/A_Star.gif" alt="animated" />
</p>

Here is a great video from the *Computerphile* YouTube channel that explains A* quite well: https://www.youtube.com/watch?v=ySN5Wnu88nE

#### (1.1.4) Heuristic State-Space Search

While we could apply the uninformed search algorithms to any search tree (and thus to any classical planning problem), we would need to define the heuristic first if we'd want to apply heuristic search methods like A* to classical planning problems. The problem here is that it is not so simple to derive heuristic functions for domain-independent planners (since these planners must solve any classical planning problems they face, within reason).

We won't dive into this topic very deeply, but it's important to understand how we can conduct heuristic state space search anyways, and to be familiar with some of the popular planners that do so. Such planners will often *relax* the problem (which we denote $P$) into a simpler problem $P'$, whose optimal solution can be found efficiently. Then, the optimal cost for solving $P'$ can be used as an heuristic for solving $P$. The length of an optimal relaxed solution is an admissible heuristic (thus allowing us to use A* to find the optimal solution for $P$), however it was shown that computing the length of an optimal relaxed solution is NP-hard (i.e. inefficient). Thus, a planner might instead just approximate the relaxed solution length for $P'$ instead of computing it exactly. Examples of planners that do this are:

* **[Heuristic Search Planner](https://bonetblai.github.io/reports/aips98-competition.pdf) (HSP):** This planner was introduced by Blai Bonet and Hector Geffner in 1997, and it relaxes a STRIPS problem by simply ignoring delete lists (i.e. negative action effects; we'll learn more about such *delete relaxed heuristics* in [next week's lecture](https://drive.google.com/open?id=1GPZOQOZjO00Hp47NXtnYLhfUMKbiU6a74PjKsNQ4STs)). It then approximates the relaxed solution length by computing weight values for all facts to estimate their distance to a certain state $s$. [Here](https://www.youtube.com/watch?v=kyCjEsFpzbA) is a YouTube video that explains HSP well.

* **[Fast Forward](http://www.cs.toronto.edu/~sheila/2542/w06/readings/ffplan01.pdf) (FF):** This planner was introduced by Joerg Hoffmann in 2001, and it is based on HSP while providing noticeable improvements. While HSP only approximates the solution length of the relaxed problem $P'$, FF obtains an explicit solution for $P'$ by using a Graphplan-style algorithm, and the number of actions in the relaxed solutions is then used as the heuristic. FF implements a novel search algorithm called Enforced Hill-Climbing (EHC), and uses the relaxed plans to prune the search space (using a method called helpful actions pruning). [Here](https://www.youtube.com/watch?v=7XH60fuMlIM) is a YouTube video that explains FF well.

* **[Fast Downward](https://www.aaai.org/Papers/JAIR/Vol26/JAIR-2606.pdf) (FD):** This planner was introduced by Malte Helmert in 2006, and its heuristic relies on building a causal graph (which we learned about in [this week's lecture](https://docs.google.com/presentation/d/12_sedRHMg1G81Usm1MppfbiooQAHPEy5vnC-HyIeiR4/edit#slide=id.g1d829a0240_1_0)) to decompose the problem hierarchically - effectively taking the computational hit up front to transform the problem into a handful of approximate but smaller problems that proves itself worthwhile in practice. [[2]](https://roboticseabass.com/2022/07/19/task-planning-in-robotics) This remains one of the more commonly used planners today, and it is integrated with the Unified Planning Framework (which we'll learn how to use in the next tutorial).

### (1.4) Alternative Classical Planning Methods

State-space search is but one of a variety of methodologies for solving classical planning problems. Some other important examples which we don't discuss here (but are discussed here and there in the lectures) are:
* Plan-Space Planning (PSP), further explained in [these lecture slides](http://www.dia.fi.upm.es/~ocorcho/Asignaturas/ModelosRazonamiento/PresentacionesClases/planning05.pdf) and [this YouTube video](https://www.youtube.com/watch?v=w3vvdlNfss4&list=PLwJ2VKmefmxpUJEGB1ff6yUZ5Zd7Gegn2&index=35&t=217s)
* Planning Graphs (for example, the greatly successful [Graphplan](https://www.cs.cmu.edu/~avrim/Papers/graphplan.pdf) algorithm proposed by Blum and Furst in 1995), which are also used in Fast Forward; further explained in [these lecture slides](https://pages.mtu.edu/~nilufer/classes/cs5811/2012-fall/lecture-slides/cs5811-ch10b-graphplan.pdf)
    * Here are some videos that explain planning graphs and Graphplan a bit more: [planning graphs 1](https://www.youtube.com/watch?v=KojBQegJbbk&list=PLwJ2VKmefmxpUJEGB1ff6yUZ5Zd7Gegn2&index=53), [planning graphs 2](https://www.youtube.com/watch?v=YPJ6yMMNx-s&list=PLwJ2VKmefmxpUJEGB1ff6yUZ5Zd7Gegn2&index=54), [Graphplan](https://www.youtube.com/watch?v=RqhSLCfZdys&list=PLwJ2VKmefmxpUJEGB1ff6yUZ5Zd7Gegn2&index=61)
* Partial-Order Planning (most notably the [Partial Order Planning Forwards (POPF)](https://planning.wiki/_citedpapers/popf2010.pdf) algorithm)
* Planning as Satisfiability, in which we represent classical planning problems as propositional satisfiability (SAT) problems and solve them using SAT solvers. This approach was proposed in [1992](https://www.cs.cornell.edu/selman/papers/pdf/92.ecai.satplan.pdf) by Kautz and Selman, but not shown to be a practical approach to planning until [1996](https://aaai.org/Papers/AAAI/1996/AAAI96-177.pdf) (also by Kautz and Selman)

Before we move on, [here](https://youtube.com/playlist?list=PLwJ2VKmefmxpUJEGB1ff6yUZ5Zd7Gegn2) is a great YouTube playlist from the University of Edinburgh that covers more in depth a lot of the topics we discussed so far and much more. If you're curious to learn more definitely check it out!

## (2) Ex: The Tower of Hanoi

Now that we have a better idea of what's going on under the hood, let's return to PDDL and practice representing more classical planning problems using it.

### (2.1) Problem Statement

The **Tower of Hanoi** is a mathematical puzzle where we have 3 pegs and $N$ disks, and initially all of the disks are stacked in decreasing value of diameter on the first peg (the smallest disk is placed on the top). The objective of the puzzle is to move the entire stack of disks to the third peg using the least number of moves (which was actually shown to be $2^{N}-1$) while obeying the following rules:
* Only one disk can be moved at a time
* Each move consists of taking the upper disk from one of the stacks and placing it on top of another stack (so a disk can only be moved if it's the uppermost disk on a stack)
* No disk may be placed on top of a smaller disk

The problem for $N=2$ looks as follows:
<p align="center">
  <img src="Graphics/Hanoi0.png" width=1000 />
</p>


### (2.2) Representing the Domain in PDDL

We begin by creating a PDDL domain file which should validly represent the domain of the Tower of Hanoi for any $N$ (the value of $N$ will be specified in the problem file - this will allow use to reuse the domain file for a wide variety of $N$ values).

First we'll declare the domain name as well as the requirements (which in this case will only be `:strips`):
```
(define (domain hanoi)
    (:requirements :strips)
```

Next we'll want to define our predicates for this domain. What would be some valuable predicates here? Well, we want to know if one disk is located on top of another disk. We also want to know if a disk has any disks on top of it (if so, then we can't move that disk). Lastly, we want to know if one disk is smaller than another (if yes, then we can place it on top of that other one should the opportunity arise). Thus, we might declare the following predicates for our domain:
```
    (:predicates (On ?x ?y)(Clear ?o)(Smaller ?a ?b))
```

Finally, we can describe the actions available in our domain. In this case, we'll only really need one action - moving a disk (`?d`) from one location (`?from`) to another (`?to`). What preconditions are necessary to conduct this action? Well, the disk we want to move must be clear, as must the location we want to move it to (whether that be another disk or a peg). The disk in question must also be originally on top of `?from`, and it must be smaller than `?to`. Applying the `move` action, we would like for the effects to be that the disk is on $?to$ and not on $?from$ anymore, that `?from` is now clear, and that `?to` is no longer clear. All in all, our action `move` could look as follows:
```
    (:action move
    :parameters (?d ?from ?to)
    :precondition (and (Clear ?d)
                       (Clear ?to)
                       (Smaller ?to ?d)
                       (On ?d ?from))
    :effect (and (On ?d ?to)
                 (Clear ?from)
                 (not (Clear ?to))
                 (not (On ?d ?from)))
    )
```

Putting it all together, our domain file should appear as follows:
```
(define (domain hanoi)
    (:requirements :strips)

    (:predicates (On ?x ?y)(Clear ?o)(Smaller ?a ?b))

    (:action move
    :parameters (?d ?from ?to)
    :precondition (and (Clear ?d)
                       (Clear ?to)
                       (Smaller ?to ?d)
                       (On ?d ?from))
    :effect (and (On ?d ?to)
                 (Clear ?from)
                 (not (Clear ?to))
                 (not (On ?d ?from)))
    )
)
```
You'll notice that we didn't define any new types here - typing is not always necessary for PDDL problems (but it will be useful for more complex ones)! You can find this domain as a `.pddl` file called `domain_hanoi.pddl` in the `Tower of Hanoi` folder.

### (2.3) Representing the Problem in PDDL

Now that our domain file has been created, we can start building problem files to represent the Tower of Hanoi problem. We will start with the $N=2$ case, where we have two disks `d1` and `d2`, along with three pegs `p1`, `p2`, `p3` (these are all of type `object` since we didn't define any new types). Adding the problem and domain names, our problem file will begin as follows:
```
(define (problem hanoi2)
        (:domain hanoi)
        
	(:objects d1 d2 p1 p2 p3)
```
All that's left now is to define the initial and goal states. In the initial state, we need to declare that the disks are smaller than all of the pegs (so that any clear disk can be moved to any clear peg), and that `d1` is smaller than `d2` - this allows us to also say that `d1` is on top of `d2` and that `d2` is on top of `p1` in the initial state. Finally, we need to state that disk `d1` and pegs `p2` and `p3` are clear. Doing so, our initial state will look as follows:
```
	(:init (On d1 d2)(On d2 p1)(Clear d1)(Clear p2)(Clear p3)(Smaller d2 d1)
        (Smaller p1 d1)(Smaller p2 d1)(Smaller p3 d1)(Smaller p1 d2)(Smaller p2 d2)(Smaller p3 d2))
```
Finally, our only requirements for the goal is that the disks should be on the third peg, i.e. `d2` is on `p3` and `d1` is on `d2`. We can write this as follows:
```
	(:goal (and (On d1 d2)(On d2 p3)))
```

Putting it all together, our problem file should appear as follows:
```
(define (problem hanoi2)
        (:domain hanoi)

	(:objects d1 d2 p1 p2 p3)
	
	(:init (On d1 d2)(On d2 p1)(Clear d1)(Clear p2)(Clear p3)(Smaller d2 d1)
        (Smaller p1 d1)(Smaller p2 d1)(Smaller p3 d1)(Smaller p1 d2)(Smaller p2 d2)(Smaller p3 d2))

	(:goal (and (On d1 d2)(On d2 p3)))
)
```
You can find this problem as a `.pddl` file called `problem_hanoi2.pddl` in the `Tower of Hanoi` folder.

### (2.4) Tower of Hanoi Solution ($N=2$)

Before we let the planner solve this problem for us, let's try to examine its state-space. Since this is a relatively small problem, we can represent the state-space graph explicitlty, and we will show the first $s$ tiers of the search tree:
<p align="center">
  <img src="Graphics/Hanoi_SS.png" width=1500 />
</p>

Notice that in the search tree we have pruned some nodes using cycle-checking (i.e. we don't repeat states in the *same* path), while we leave the state-space graph as an undirected (i.e. all edges are bidirectional) cyclic graph. We can see that the optimal solution indeed requires $s=2^{2}-1=3$ actions, and we can use a simple state-space search algorithm like BFS (given that $b\approx2$ and $s=3$, we only search $O(2^{3}=8)$ nodes - here we see that for $b\approx2$ in the worst case we would have to explore $2+4+8=14$ nodes not including the root node) to solve this problem. For decently higher values of $N$, though, BFS becomes quickly inefficient. Say we have 64 disks - the optimal solution would require $s=2^{64}-1=18,446,744,073,709,551,615\approx1.845\cdot10^{19}$ moves to complete. Assuming you could move the disks at a rate of one per second, it would take you about 585 billion years to finish the puzzle, which is roughly 42 times the current age of the universe. With a solution this huge, just imagine trying to conduct a state-space search on it - BFS would have to explore $\sum_{i=1}^{2^{64}-1}2^{i}=2(2^{2^{64}-1}-1)\approx O(2^{2^{64}})$ nodes in the worst case!

Back to PDDL, our planner (I'm using the VS Code extension for PDDL, which in this case chose an algorithm called [Iterative-Width (IW)](https://people.eng.unimelb.edu.au/nlipovetzky/papers/classical-width-ecai12.pdf) Search) returned the following plan in less than one second for the $N=2$ Tower of Hanoi problem:
```
(move d1 d2 p2)
(move d2 p1 p3)
(move d1 p2 d2)
```
And this is exactly the sequence we would've expected. We can also use the same domain file and the `problem_hanoi3.pddl` in the `Tower of Hanoi` folder to solve the $N=3$ case, which will return the following plan:
```
(move d1 d2 p3)
(move d2 d3 p2)
(move d1 p3 d2)
(move d3 p1 p3)
(move d1 d2 p1)
(move d2 p2 d3)
(move d1 p1 d2)
```
We see that the optimal plan has $s=2^{3}-1=7$ steps, as expected. As a home exercise, see that you can produce PDDL files for the Tower of Hanoi problem with $N>3$, and see how long it takes your solver to reach the optimal solution (try to solve it yourself as well, and compare your performance to that of the PDDL solver).

## (3) Ex: The Travelling Jedi Problem

For this exercise, you will try to model the Travelling Jedi Problem (which is just the unweighted [Travelling Salesman Problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem)) yourselves in class using PDDL! I would like for you to also try to draw the search tree for this problem, and look at how different search algorithms would solve it (you can assign arbitrary costs and/or heuristics to the edges as you'd like for all methods other than BFS and DFS). I will give you 10-15 minutes to do so, good luck!

### (3.1) Problem Statement

Luke is taking a roadtrip with his pals aboard the Millennium Falcon, and he wants to visit a bunch of different planets, moons, and space stations across the galaxy. He will start from Tatooine, and he wants to visit every location exactly once before returning to Tatooine again. He asks the Falcon's computer (known as the Millennium Collective) to plan such a route for him, without regard to the actual geometric distance between the locations (we will deal with a weighted version of the TSP later on). The following figure shows the locations Luke would like to visit, as well as the routes that connect them to each other:
<p align="center">
  <img src="Graphics/StarWarsTJP.png" width=600 />
</p>

**Nerdy Digression:** Note that Luke wants to visit the Forest Moon of Endor and *not* the planet Endor (which is a gas giant where the Ewoks surely wouldn't have a very nice time), and that he wants to visit the Death Star and *not* Alderaan (unfortunately exploded planets don't make for great vacation spots).

### (3.2) Solution

The solution files can be found in the folder called `Travelling Jedi Problem`. The search tree of the problem looks as follows:
<p align="center">
  <img src="Graphics/TJP_SearchTree.png" width=600 />
</p>

I didn't add the nodes that terminate at Tatooine without passing through all of the other locations first, for the sake of keeping the image cleaner. We can see that the maximum depth $m$ is equal to the shallowest solution depth $s$ (there is no way to travel to all 7 locations exactly once in more than 7 moves), and so $m=s=7$. There are 43 nodes in this tree, so an algorithm like BFS will return the optimal solution (since the costs of all edges are uniform) quite quickly.

## (4) Conclusion

In this tutorial, we:
* Introduced some classical planning algorithms, with a focus on state-space search methods
* Covered some more classical planning problems like the Tower of Hanoi and the Travelling Salesman Problem
* Gave you your first (probably) hands-on experience writing PDDL files

Next week, we will begin our discussion of temporal planning and scheduling, and we'll learn how to use the Unified Planning Framework (UPF) library in Python to easily write and solve PDDL problems using Python scripts.

#### ***Credit:** This tutorial was written by Yotam Granov, Winter 2022.*

### **References**

[1] Inteligencia Futura. ["Games, Routes and Circuits, Problem Solving with Artificial Intelligence"](https://inteligenciafutura.mx/english-version-blog/games-routes-and-circuits-problem-solving-with-artificial-intelligence). 2019.

[2] Robotic Sea Bass. ["Task Planning in Robotics"](https://roboticseabass.com/2022/07/19/task-planning-in-robotics/). 2022. 

[3] Oren Salzman & Sarah Keren. Technion "Introduction to Artificial Intelligence" Course Lectures on Blind Search and Heuristic Search. 2022.

[4] Stuart Russell & Peter Norvig. ["Artificial Intelligence: A Modern Approach, Third Edition"](https://zoo.cs.yale.edu/classes/cs470/materials/aima2010.pdf). *Pearson Education*, 2010.

[5] Malik Ghallab, Dana Nau & Paolo Traverso. ["Automated Planning and Acting"](https://projects.laas.fr/planning/). *Cambridge University Press*, 2016.